Em 2018 ajudei a Mariem Hakobyan com a sua tese de licenciatura. O projecto envolveu separar a voz e a guitarra de faixas de música de fado. Usámos Non-Negative Matrix Factorization para separar as fontes. Inicialmente, esperavamos que a técnica de separação de fontes funcionasse bem o suficiente para ajudar no topic modelling, mas não foi o caso.

In [7]:
# nmf.py

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy import signal
from scipy.io import wavfile
import numpy as np
import pandas as pd
from sklearn.decomposition import NMF
import os

workingDir = "/ffs/tmp/frsc/fado/audio_separation/"
#nData = 117
nData = 10


In [10]:
!ls {workingDir}

100GuitarSpectrograms.npy  GuitarSpectrograms.npy  NewGuitar3.wav
20GuitarSpectrograms.npy   labels.txt		   NMF
guitarMusic		   NewBook1test.wav	   NoisyGuitar1.wav
GuitarNMF_H.npy		   NewGuitar1.wav	   speech
GuitarNMF_W.npy		   NewGuitar2.wav


In [11]:
spectrograms = np.load(workingDir + "GuitarSpectrograms.npy")
spectrograms=spectrograms.T

In [23]:
spectrograms_df = pd.DataFrame(spectrograms)
spectrograms_df = spectrograms_df[:3]

In [24]:
spectrograms_df.describe()

,0,1,2,3,4,5,6,7,8,9,...,1328423,1328424,1328425,1328426,1328427,1328428,1328429,1328430,1328431,1328432
count,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
mean,81.715621,135.645146,197.440525,-29.240940,-55.419218,-20.811165,-53.944400,-116.804188,-105.247466,-2.881079,...,-0.119868,0.247443,-0.026736,0.340784,0.027438,0.153801,-0.243131,-0.255996,0.014104,0.136837
std,191.405776,212.831774,308.948235,368.425491,292.730864,405.858322,120.874071,383.785799,418.866773,308.633216,...,0.447670,0.159456,0.582354,0.095461,0.238868,0.364367,0.716266,0.242028,0.514899,3.566002
min,-55.270128,-0.012945,0.002820,-411.416202,-371.902182,-436.675099,-192.397929,-545.418619,-566.701439,-312.942389,...,-0.555979,0.093251,-0.388990,0.231026,-0.246009,-0.060941,-0.793791,-0.473389,-0.543100,-2.575891
25%,-27.634661,12.996434,19.421882,-205.707394,-185.945842,-218.337115,-96.198236,-272.714310,-283.349595,-156.473587,...,-0.349065,0.165321,-0.362614,0.308935,-0.056559,-0.056551,-0.648012,-0.386392,-0.215007,-1.882725
50%,0.000807,26.005812,38.840944,0.001414,0.010497,0.000869,0.001457,-0.010000,0.002248,-0.004786,...,-0.142151,0.237391,-0.336238,0.386845,0.132891,-0.052160,-0.502233,-0.299396,0.113085,-1.189559
75%,150.208495,203.474191,296.159377,161.846691,102.822264,187.120802,15.282365,97.503028,125.479521,152.149576,...,0.098188,0.324539,0.154390,0.395663,0.164162,0.261173,0.032198,-0.147300,0.292706,1.493201
max,300.416183,380.942571,553.477810,323.691968,205.634030,374.240734,30.563273,195.016057,250.956794,304.303938,...,0.338527,0.411687,0.645018,0.404481,0.195433,0.574506,0.566630,0.004796,0.472327,4.175961


In [ ]:
#spectrograms = np.genfromtxt(open(workingDir + "GuitarSpectrograms.csv", "rb"), delimiter="," , dtype=complex)
spectrograms = np.load(workingDir + "GuitarSpectrograms.npy")

absolute_spectrogram = np.abs(spectrograms[:,:nData])

#for i in range(nData):
model = NMF(n_components=9,init='nndsvdar', beta_loss=1, solver='mu')
print("antes W")
X = absolute_spectrogram

W = model.fit_transform(X)
print("antes H")
H = model.components_

np.save(workingDir + 'NMF/' + 'W/' + '10GuitarNMF_W', W)
np.save(workingDir + 'NMF/' + 'H/' + '10GuitarNMF_H', H)

#TODO:tentar inverter e obter som dos NMF


In [2]:
# istft.py

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy import signal
from scipy.io import wavfile
import numpy as np
from sklearn.decomposition import NMF
import os

nData = 141
workingDir = "/ffs/tmp/frsc/fado/audio_separation/"


In [ ]:




W = np.zeros(shape=(nData, 2585, 100))
H = np.zeros(shape=(nData, 100, 513))


for i in range(nData):
    print("loading W")
    W[i,:,:] = np.load(workingDir + 'NMF/' + 'W/' + 'GuitarNMF_W' + str(i) + '.npy')
    print("loading H")
    H[i,:,:] = np.load(workingDir + 'NMF/' + 'H/' + 'GuitarNMF_H' + str(i) + '.npy')

print("W:" + str(W.shape) + " - H:" + str(H.shape))

songn = 2;
newSpectrogram = np.dot(W[songn - 1], H[songn - 1]).T


print("newSpectrogram pre reshape:" + str(newSpectrogram.shape))
#newSpectrogram = np.reshape(newSpectrogram, spectrogramShape)
print("newSpectrogram post reshape:" + str(newSpectrogram.shape))


print("writing wav")
#INVERSE FOURIER
newTimes, newMusic = signal.istft(newSpectrogram, fs = sample_rate, nperseg=segmentSize, noverlap=(segmentSize/2))

print("newMusic:" + str(newMusic.shape))

oLength = newMusic.size
inverseMusic = np.int16(np.round(newMusic))

wavfile.write(workingDir + "/NewGuitar" + str(songn) + ".wav", sample_rate, inverseMusic)
